In [1]:
import os
os.environ["WANDB_DISABLED"] = "true"

import sys
sys.path.append('..')

%load_ext autoreload
%autoreload 2

In [36]:
import analyze_utils

# https://wandb.ai/jack-morris/emb-inv-1/runs/ebb31d91810c4b62d2b55b5382e8c7ea/logs?workspace=user-jxmorris12
checkpoint_folder = '/home/jxm3/research/retrieval/inversion/saves/db66b9c01b644541fedbdcc59c53a285/ebb31d91810c4b62d2b55b5382e8c7ea'
args_str = '--per_device_train_batch_size 128 --per_device_eval_batch_size 128 --max_seq_length 32 --model_name_or_path t5-base --embedder_model_name gtr_base --num_repeat_tokens 16 --embedder_no_grad True --exp_group_name mar17-baselines --learning_rate 0.0003 --freeze_strategy none --embedder_fake_with_zeros False --use_frozen_embeddings_as_input False --num_train_epochs 24 --max_eval_samples 500 --eval_steps 25000 --warmup_steps 100000 --bf16=1 --use_wandb=1'
trainer = analyze_utils.load_inversion_model_and_trainer(checkpoint_folder, args_str)

trainer.evaluate()

[0] Loading model from checkpoint: /home/jxm3/research/retrieval/inversion/saves/db66b9c01b644541fedbdcc59c53a285/ebb31d91810c4b62d2b55b5382e8c7ea/checkpoint-999744
Set train_args.dataloader_num_workers = 4
[1] creating model & stuff


[2] tokenizing dataset & preprocessing embeddings
[3] initializing trainer
[4] getting ckpnt
[5] loading ckpt /home/jxm3/research/retrieval/inversion/saves/db66b9c01b644541fedbdcc59c53a285/ebb31d91810c4b62d2b55b5382e8c7ea/checkpoint-999744


to the character of the skull, which are relatively smooth and untutored in the case of infant sutures. Nevertheless, the sutures
to the character of the sutures of the skull which, like those of the infant skull, are relatively smooth and untortuous. In fact



individual from the Southern Hemisphere to win the Winter Olympic relay gold medal, and was also part of the Australian Short Track team which won a
individual from the Southern Hemisphere, to win a Winter Olympic gold medal and was also part of the short track relay team that won Australia'



the same rights as men, and 75% agreed that they should be protected from discrimination. Among the other 15% are heterosexuals, no work
the same rights as straight people, while 15% disagreed. Additionally, 69% agreed that they should be protected from workplace discrimination. 13% of H


{'eval_loss': 1.0522304773330688,
 'eval_bleu_score': 31.552541624779003,
 'eval_accuracy': 0.7384440104166666,
 'eval_perplexity': 2.8640321578446315,
 'eval_runtime': 16.2888,
 'eval_samples_per_second': 30.696,
 'eval_steps_per_second': 0.246}

In [37]:
import torch
from tqdm.notebook import trange

trainer.args.per_device_eval_batch_size = 128

d = trainer.get_eval_dataloader()
batch = next(iter(d))
batch['input_ids'].shape

torch._dynamo.config.verbose = True
trainer.model.eval()

gen_kwargs = trainer.gen_kwargs
# gen_kwargs["temperature"] = 1e-18
# gen_kwargs["do_sample"] = True

torch.set_grad_enabled(False)

input_embs = trainer.model.call_embedding_model(
    input_ids=batch['embedder_input_ids'].cuda(),
    attention_mask=batch['embedder_attention_mask'].cuda(),
)

all_data = []
outputs = trainer.model.generate(
    inputs={k: v.cuda() for k,v in batch.items()},
    generation_kwargs=gen_kwargs
)

In [47]:
batch['input_ids'][2]

tensor([    8,   337,  2166,    38,  2541,   151,     6,   298, 13914, 15788,
           26,     5,  5433,     6,   431,  7561,  4686,    24,    79,   225,
           36,  5046,    45,  6940,  9192,   257,     5,   209,  5170,    13,
          454,     1])

In [55]:
outputs

tensor([[    0,    12,     8,  ...,     8,  2629, 10471],
        [    0,   928,    45,  ...,   751,     3,     9],
        [    0,     8,   337,  ...,     6,   150,   161],
        ...,
        [    0,    28,     3,  ...,   731,  2812,  2069],
        [    0,  2144,  2345,  ...,   121,  2144,  2345],
        [    0,    13,  7482,  ...,     6,   186,   999]], device='cuda:0')

tensor([[   12,     8,  1848,  ...,  2629, 10471,     1],
        [  928,    45,     8,  ...,     3,     9,     1],
        [    8,   337,  2166,  ...,   150,   161,     1],
        ...,
        [   28,     3,     9,  ...,  2812,  2069,     1],
        [ 2144,  2345,    41,  ...,  2144,  2345,     1],
        [   13,  7482,   291,  ...,   186,   999,     1]], device='cuda:0')

## checking likelihood of generated text

In [62]:
# fix bos/eos tokens to align between input_ids and outputs
new_outputs = torch.cat((outputs[:, 1:], torch.ones((len(outputs), 1), dtype=outputs.dtype, device=outputs.device)), dim=1)

In [70]:
torch.set_grad_enabled(False)

inputs_embeds, attention_mask = trainer.model.embed_and_project(
    embedder_input_ids=batch["embedder_input_ids"].cuda(),
    embedder_attention_mask=batch["embedder_attention_mask"].cuda(),
)
true_outputs = trainer.model.encoder_decoder(
    inputs_embeds=inputs_embeds,
    attention_mask=attention_mask,
    labels=batch["input_ids"].cuda(),
)
pred_outputs = trainer.model.encoder_decoder(
    inputs_embeds=inputs_embeds,
    attention_mask=attention_mask,
    labels=new_outputs,
)

In [73]:
import math

print(f"true data has ppl under model: {math.exp(true_outputs.loss)}")
print(f"sampled data has ppl under model: {math.exp(pred_outputs.loss)}")

true data has ppl under model: 2.8347514997989016
sampled data has ppl under model: 1.7082817983083902


## checking with sampling

In [2]:
gen_kwargs = trainer.gen_kwargs

def sample_text_with_noise(embedder_input_ids: torch.Tensor, embedder_attention_mask: torch.Tensor, alpha: float):
    inputs_embeds, attention_mask = trainer.model.embed_and_project(
        embedder_input_ids=embedder_input_ids.cuda(),
        embedder_attention_mask=embedder_attention_mask.cuda(),
    )
    inputs_embeds += torch.randn(inputs_embeds.shape, device=inputs_embeds.device) * alpha
    return trainer.model.encoder_decoder.generate(
        inputs_embeds=inputs_embeds,
        attention_mask=attention_mask,
        **gen_kwargs,
    )

sample_text_with_noise(
    embedder_input_ids=batch["embedder_input_ids"],
    embedder_attention_mask=batch["embedder_attention_mask"],
    alpha=0.0,
)

NameError: name 'trainer' is not defined